In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [3]:
reviews = pd.read_csv('Reviews.csv')

In [4]:
print (reviews)


              Timestamp  How would you score the class   \
0    2/15/2019 13:42:01                               1   
1    2/15/2019 14:37:29                               1   
2    2/15/2019 11:06:42                               2   
3    2/15/2019 11:08:46                               2   
4    2/15/2019 11:06:05                               3   
5    2/15/2019 11:06:52                               3   
6    2/15/2019 11:07:21                               3   
7    2/15/2019 11:07:34                               3   
8    2/15/2019 11:09:17                               3   
9    2/15/2019 11:10:14                               3   
10   2/15/2019 11:56:54                               3   
11   2/15/2019 13:07:27                               3   
12   2/15/2019 13:40:02                               3   
13   2/15/2019 13:40:37                               3   
14   2/15/2019 14:38:11                               3   
15   2/15/2019 15:30:21                               3 

In [5]:
reviews.head()

,Timestamp,How would you score the class,Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?),"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [6]:
reviews.shape


(133, 8)

In [7]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 8 columns):
Timestamp                                                                                                                          133 non-null object
How would you score the class                                                                                                      133 non-null int64
Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)    132 non-null object
Polarity (-1, 0, 1)                                                                                                                0 non-null float64
Suggestions (0 or 1)                                                                                                               0 non-null float64
Suggestions                                                                                                                        0 non-null fl

In [8]:
reviews.describe()


,How would you score the class,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
count,133.000000,0.0,0.0,0.0,0.0,0.0
mean,4.255639,NaN,NaN,NaN,NaN,NaN
std,0.813281,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,4.000000,NaN,NaN,NaN,NaN,NaN
50%,4.000000,NaN,NaN,NaN,NaN,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN
max,5.000000,NaN,NaN,NaN,NaN,NaN


In [9]:
reviews.rename(
    columns={
        'Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)' : 'rev', 'How would you score the class ' : 'stars'
    },
    inplace=True
)
reviews.head()

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [10]:
reviews['rev'] = reviews['rev'].apply(str)
## This converts the column values of reviews to strings

In [11]:
reviews['text length'] = reviews['rev'].apply(len)
reviews.head()

#this creates a new column that will apply the text length of review

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data,text length
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN,167
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN,86
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN,117
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN,268
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN,113


In [12]:
X = reviews['rev']
y = reviews['stars']

In [32]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
one_hot_results = tokenizer.texts_to_matrix(X, mode='binary')
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))
print(tokenizer)
print(one_hot_results)
print(word_index)


Found 657 unique tokens.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'and': 1, 'i': 2, 'is': 3, 'class': 4, 'the': 5, 'to': 6, 'we': 7, 'a': 8, 'this': 9, 'it': 10, 'because': 11, 'fun': 12, 'like': 13, 'that': 14, 'of': 15, 'you': 16, 'have': 17, 'good': 18, 'but': 19, 'also': 20, 'get': 21, 'mr': 22, 'are': 23, 'notes': 24, 'more': 25, 'very': 26, 'in': 27, 'do': 28, 'us': 29, 'liberman': 30, 'teacher': 31, 'really': 32, 'not': 33, 'lot': 34, 'learn': 35, 'should': 36, 'people': 37, 'think': 38, "it's": 39, 'too': 40, 'time': 41, "don't": 42, 'give': 43, 'better': 44, 'so': 45, 'my': 46, 'could': 47, 'homework': 48, 'take': 49, 'he': 50, 'gave': 51, 'much': 52, 'our': 53, 'some': 54, 'one': 55, 'need': 56, 'with': 57, 'other': 58, 'make': 59, 'projects': 60, 'its': 61, 'score': 62, 'for': 63, 'things': 64, 'many': 65, 'if': 66, 'bad': 67, 'makes': 68, 'favorite': 69, 'work

In [20]:
print(bow_transformer.vocabulary_)


{'we': 611, 'suck': 526, 'no': 377, 'naptime': 371, 'cheese': 101, 'ohnononononnononononononononononnononononononononononononoonononno': 392, 'because': 67, 'of': 390, 'mr': 364, 'liberman': 320, 'and': 41, 'i': 256, 'need': 372, 'to': 575, 'calm': 89, 'down': 149, 'have': 238, 'leavened': 312, 'bread': 84, 'unleavened': 592, 'better': 74, 'grades': 225, 'gave': 208, 'this': 567, 'class': 107, 'a': 13, '2': 6, 'it': 285, 'is': 283, 'too': 577, 'fast': 185, 'for': 196, 'everyone': 176, 'catch': 94, 'up': 595, 'barely': 63, 'any': 48, 'time': 572, 'communicate': 112, 'with': 631, 'each': 150, 'other': 402, 'also': 34, 'you': 652, 'scammed': 477, 'us': 596, 'pizza': 424, 'think': 566, 'should': 488, 'stop': 512, 'giving': 218, 'so': 499, 'much': 366, 'work': 635, 'even': 173, 'though': 570, "don't": 145, 'homework': 249, "it's": 286, 'still': 511, 'not': 381, 'good': 221, 'fun': 205, 'teacher': 545, 'but': 85, 'right': 471, 'write': 641, 'essays': 172, 'whcih': 620, 'make': 346, 'skids': 

In [18]:
print(bow_transformer.transform([X[24]]))

  (0, 13)	2
  (0, 41)	1
  (0, 44)	1
  (0, 52)	1
  (0, 67)	1
  (0, 76)	1
  (0, 107)	3
  (0, 152)	1
  (0, 176)	1
  (0, 205)	1
  (0, 221)	1
  (0, 238)	1
  (0, 256)	1
  (0, 283)	4
  (0, 311)	1
  (0, 396)	1
  (0, 397)	1
  (0, 402)	1
  (0, 442)	1
  (0, 467)	1
  (0, 517)	1
  (0, 554)	1
  (0, 555)	1
  (0, 557)	1
  (0, 562)	1
  (0, 567)	1
  (0, 631)	1


In [19]:
print (X[24])

The class is a good class, everyone is respectful. I only have problems with one student because they are a bit annoying. Other than that this class is fun and learning is easy.
